In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
import csv
import numpy as np
from bs4 import BeautifulSoup
import schedule
import time
from datetime import datetime
#Thư viện để dừng schedule
import sys
import os
import signal
#thu vien thread chay da luong
import threading
#cai dat va chay driver chrome
url = "https://iboard.ssi.com.vn/bang-gia/hose"
driver = webdriver.Chrome("E:\chromedriver_win32\chromedriver.exe")
driver.get(url)
#Đọc các mã chứng khoán trong file Ma_chung_khoan.txt
Ma_chung_khoan=open("Ma_chung_khoan.txt","r")
Ma_CK=Ma_chung_khoan.readlines()
Ma_chung_khoan.close() 
#loại bỏ dấu  ","
Ma_CK=Ma_CK[0].split(',')
for mck in Ma_CK:
    #Đường link file cào, sys.path[0] là đường dẫn thư mục của code         
    save_path = sys.path[0].replace('\\','/')+'/DATA_HOSE/'+mck
    #Kiểm tra xem đã có folder đó chưa, nếu chưa thì tạo mới
    if not os.path.exists(save_path):
        os.makedirs(save_path)



In [2]:
#Xử lí lưu file
def xu_ly_file(a,mck):
        #Đường link file cào, sys.path[0] là đường dẫn thư mục của code 
        save_path = sys.path[0].replace('\\','/')+'/DATA_HOSE/'+mck
        #Tên file.csv: Mã chứng khoán + ngày tháng năm
        completeName = os.path.join(save_path,mck+"_"+datetime.today().strftime('%d_%m_%Y')+".csv")
        #sửa mode w+ thành mode a để viết được vào cuối file
        file = open(completeName,"a",)
        with file:
            write = csv.writer(file)
            write.writerows(np.array(a).reshape(-1,27))
#Crawl Mã
def job():
    start_time = time.time()
    p=str(datetime.now())
    #download sourse html page
    page_source = BeautifulSoup(driver.page_source,"lxml")
    print("--- %s seconds ---" % (time.time() - start_time))    
    for mck in Ma_CK:
        a=[]
        info_tr=page_source.find('tr', id=mck)
        info_td=info_tr.findAll('td')
        a.extend([p])
        for i in range(26):
            if info_td[i].get_text()=='':
                a.extend("N")
            else:
                a.append(info_td[i].get_text())
        xu_ly_file(a,mck)

In [3]:
#lớp cào chia luồng
class crawler(threading.Thread):
    
    def __init__(self, list_lock):
        threading.Thread.__init__(self)
        self._list_lock = list_lock	# Lock for list_num

    def run(self):
        while True:
        	# request to access shared resource
            # if there are many threads acquiring Lock, only one thread get the Lock
            # and other threads will get blocked
            #lấy khóa
            self._list_lock.acquire()
            job()
            #trả khóa
            self._list_lock.release()
    
    
def parallel_primes(num_threads=None):
    """Parallel count number of prime from 2 to n

    Count prime numbers from 2 to n using num_threads threads
    If num_threads is None, using os.cpu_count()

    """
    list_lock = threading.Lock()
    threads = []

    if num_threads is None:
        try:
            num_threads = os.cpu_count()
        except AttributeError:
            num_threads = 4

    elif num_threads < 1:
        raise ValueError('num_threads must be > 0')

    for i in range(num_threads):
        thread = crawler(list_lock)
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()


schedule.every().day.at("11:19").do(parallel_primes)
schedule.every().day.at("11:20").do(exit)
#vòng while này kiểm tra mỗi giây coi đúng với tgian mình muốn để thực hiện hay chưa
while True:
    schedule.run_pending()
    time.sleep(1)

--- 1.026102066040039 seconds ---
--- 1.1344316005706787 seconds ---
--- 1.2093513011932373 seconds ---
--- 1.1006906032562256 seconds ---
--- 1.1081418991088867 seconds ---
--- 1.0639183521270752 seconds ---
--- 0.9829506874084473 seconds ---
--- 0.993927001953125 seconds ---
--- 1.0438547134399414 seconds ---
--- 0.9550397396087646 seconds ---
--- 0.9798800945281982 seconds ---
--- 1.1005442142486572 seconds ---
--- 1.1116485595703125 seconds ---
--- 1.1020431518554688 seconds ---
--- 1.071878433227539 seconds ---
--- 1.0980491638183594 seconds ---
--- 0.9899885654449463 seconds ---
--- 1.0387094020843506 seconds ---
--- 1.1811795234680176 seconds ---
--- 1.1612200736999512 seconds ---
--- 0.9964473247528076 seconds ---
--- 1.0009119510650635 seconds ---
--- 1.0742247104644775 seconds ---
--- 1.118668556213379 seconds ---
--- 1.135122537612915 seconds ---
--- 1.2116506099700928 seconds ---
--- 0.9559900760650635 seconds ---
--- 1.0632145404815674 seconds ---
--- 1.046816349029541 sec